In [14]:
# Import myfunc at cix folder
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../cix')
import myfuncs as mf
import pandas as pd
from rdkit.Chem import rdBase, RDConfig
from rdkit import Chem
rdBase.DisableLog('rdApp.*') # To make rdkit silent
from rdkit.Chem import PandasTools as pt
from rdkit.Chem import Descriptors
import numpy as np
import chemfp
import csv
import time
import os

The purpose of this script is to analyze the output of the CMD trained with a diverse set of 300K dissimilar ("orthogonal") compounds as created by the divsamp0 function and compare it with the output of the CMD trained with a clustered set of 300K compounds. As source of compounds, the set of ca. 5 million clean lead-like compounds from ZINC12 is used. From it, a collection of 300K orthogonal molecules were previously sampled by means of divsamp0 (see Exp4DivSamp script). 

The correctness, diversity and novelty of the output obtained with the orthogonal training set is compared with the output of the random training set. The correctness of the training and output files is assessed by the percentage of correct SMILES. The diversity of the training and output files are assessed by counting the number of clusters, frames and generic frames in both sets. The novelty of the training set is assessed by the percentage of molecules with a Tanimoto similarity < 0.7 to any molecule in the training set, and the percentage of frames or generic frames not present in the training set.

In an initial step, the orthogonality of the orthogonal set is checked. 

In [15]:
##########################################
## Check orthogonality of input smiles
##########################################

# Init the time counter for the whole notebook
start = time.time()


smis = mf.smif2smis('./divtrain300000.smi')
ncorr, n, smis, wrongsmis = mf.corrsmis(smis)
smidf = mf.smis2smidf(smis)
ar = mf.smidf2arena(smidf)

In [16]:
sim_t = search.threshold_tanimoto_search_symmetric(ar, threshold=0.75)

# Sum of neighbors in the similarity table
sum([len(indices) for (i,indices) in enumerate(sim_t.iter_indices())])

1801076

In [17]:
##########################################################
## Analysis of the unconditioned output - diverse set
##########################################################

# Init the time counter for the whole notebook
start = time.time()


# 2D plot of MW vs logP of training set
data_uri='../div0le.csv'
Y = pd.read_csv(data_uri)
Y = Y[["mw", "logp"]]
mf.bidiplot(np.asarray(Y), "MWt","LogP", d = True)

IOError: File ../div0le.csv does not exist

In [ ]:

it = range(100000, 300001, 100000)

df_un_d, cls_un_d = mf.wholean(it = it, name_train = "div0letrain", name_pref = "div0leunc")

In [ ]:
# Show the results in the output dataframe

df_un_d

In [ ]:
# Save the results

df_un_d.to_csv("analysis1-div-un.csv")

In [ ]:
# Plot the clusters distributions and cluster size distribution
mf.plotmulticlus(cls_un_d, 10, 5)

In [ ]:
##########################################################
## Analysis of the conditioned output - diverse set
##########################################################

df_co_d, cls_co_d = mf.wholean(it = it, name_train = "div0letrain", name_pref = "div0lecon")

In [ ]:
# Show the results dataframe

df_co_d

In [ ]:
# Save the results dataframe

df_co_d.to_csv("analysis1-div-co.csv")

In [ ]:
# Plot the clusters distributions and cluster size distribution

mf.plotmulticlus(cls_co_d, 10, 5)

In [ ]:
### Plot the bunch of histograms of mwt

mwts = []

for n in it:
    smis = mf.smif2smis('./div0lecon' + str(n) + '.smi')
    ncorr, n, smis, wrongsmis = mf.corrsmis(smis)
    smidf = mf.smis2smidf(smis)
    pt.AddMoleculeColumnToFrame(smidf,"smiles")
    smidf['mw'] = smidf['ROMol'].map(Descriptors.MolWt)
    del smidf["ROMol"]
    mwts.append(list(smidf['mw']))

leg = ["# train=300K"] 

mf.paintmultihist([mwts], "MWt", 1, 1, 270, 300, 5, 5, 210, 400, leg)

In [ ]:
##########################################################
## Analysis of the unconditioned output - clustered set
##########################################################

it = range(50000, 300001, 50000)

df_un_c, cls_un_c = mf.wholean(it = it, name_train = "clutrain", name_pref = "cluunc")

In [ ]:
# Show the results in the output dataframe

df_un_c

In [ ]:
# Save the results

df_un_c.to_csv("analysis1-clu-un.csv")

In [ ]:
# Plot the clusters distributions and cluster size distribution
mf.plotmulticlus(cls_un_c, 10, 5)

In [ ]:
##########################################################
## Analysis of the conditioned output - clustered set
##########################################################

df_co_c, cls_co_c = mf.wholean(it = it, name_train = "clutrain", name_pref = "clucon")

In [ ]:
# Show the results dataframe

df_co_c

In [ ]:
# Save the results dataframe

df_co_c.to_csv("analysis1-clu-co.csv")

In [ ]:
# Plot the clusters distributions and cluster size distribution

mf.plotmulticlus(cls_co_c, 10, 5)

In [ ]:
### Plot the bunch of histograms of mwt
it = range(300000, 300001)
mwts = []

for n in it:
    smis = mf.smif2smis('./clucon' + str(n) + '.smi')
    ncorr, n, smis, wrongsmis = mf.corrsmis(smis)
    smidf = mf.smis2smidf(smis)
    pt.AddMoleculeColumnToFrame(smidf,"smiles")
    smidf['mw'] = smidf['ROMol'].map(Descriptors.MolWt)
    del smidf["ROMol"]
    mwts.append(list(smidf['mw']))

leg = ["# train=300K"] 

mf.paintmultihist(mwts, "MWt", 1, 1, 270, 300, 5, 5, 210, 400, leg)

In [ ]:
# End the time counter for the whole notebook
end = time.time()
eltime = end - start
print('Exp4nalysis execution time: ' + time.strftime("%H:%M:%S", time.gmtime(eltime)))